<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

In [243]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 57 (delta 32), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (57/57), 385.92 KiB | 4.71 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo/cloned-repo
dataset_original.py  EAST_dataset.ipynb        loss.py	 train.py
dataset.py	     EAST_text_detector.ipynb  model.py
CPU times: user 116 ms, sys: 209 ms, total: 325 ms
Wall time: 14.2 s


In [244]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [245]:
from dataset_original import custom_dataset

image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Images'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/data/ICDAR dataset/Labels'

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

10


In [0]:
def extract_vertices(lines):
	'''extract vertices info from txt lines
	Input:
		lines   : list of string info
	Output:
		vertices: vertices of text regions <numpy.ndarray, (n,8)>
		labels  : 1->valid, 0->ignore, <numpy.ndarray, (n,)>
	'''
	labels = []
	vertices = []
	for line in lines:
		vertices.append(list(map(int,line.rstrip('\n').lstrip('\ufeff').split(',')[:8])))
		label = 0 if '###' in line else 1
		labels.append(label)
	return np.array(vertices), np.array(labels)

(array([[377, 117, 463, 117, 465, 130, 378, 130],
        [493, 115, 519, 115, 519, 131, 493, 131],
        [374, 155, 409, 155, 409, 170, 374, 170],
        [492, 151, 551, 151, 551, 170, 492, 170],
        [376, 198, 422, 198, 422, 212, 376, 212],
        [494, 190, 539, 189, 539, 205, 494, 206],
        [374,   1, 494,   0, 492,  85, 372,  86]]),
 array([1, 1, 0, 1, 1, 0, 0]))

In [0]:
from dataset_original import *

In [272]:
img = Image.open(image_address+'/img_1.jpg')

with open(label_address+'/gt_img_1.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)

[261.94353436  64.1845955  347.93853214  65.11214936 349.77663316
  80.1328479  262.78169354  79.19450853]
--------------------
[377.95835821  63.43583099 403.95684591  63.71625425 403.76270673
  81.71520728 377.76421902  81.43478402]
--------------------
[258.47993193 107.14973786 293.47789614 107.52723071 293.30532798
 123.52630007 258.30736377 123.14880722]
--------------------
[376.52699597 103.42271887 435.52356422 104.05906396 435.29706851
 125.05784249 376.30050026 124.4214974 ]
--------------------
[259.96211111 155.16851695 305.95943551 155.66465041 305.79765286
 170.66377793 259.80032846 170.16764447]
--------------------
[378.0523172  147.44173062 423.06048527 146.92713673 422.8771316
 163.92614792 377.85817802 165.44068364]
--------------------
[260.33503965 -64.8402577  380.33884532 -64.54593834 377.32512371
  29.4270231  257.31053253  30.13264557]
--------------------


In [273]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(142.)